# Demo - Exact Matching

In [1]:
# Sets up the location of the api relative to this notebook 
import sys
sys.path.append('../../')

In [2]:
# Import the module for connection to a sqllite database
from esg_matching.engine.connectors.sql_lite import SqlLiteConnector

In [3]:
# Import the modules for file management
from esg_matching.file_reader.file import File
from esg_matching.file_reader.csv_reader import FileReaderCsv

In [4]:
# Import the modules for the etl processing: reading, transformation and loading data to a database
from esg_matching.processing.etl import EtlProcessing

## 1. Database setup

In [5]:
# Localization of the database to be created in relation to this jupyter notebook
# The database will be created in the /data/dabase folder, under the project main folder
path_db = '../../tests/data/demo/test_esg_matching_CLEAN.db'

In [6]:
# The database connector is represented by the class SqlLiteConnector 
db_conn = SqlLiteConnector()

In [7]:
# The connect() method of the SqlLiteConnector is used to stablish a connection with the database if it exists, 
# or to create a new one. The property path_db defines the location and name of the database.
# The  property show_sql_statement indicates if the SQL statements are echoed (or printed) in the default output channel.
db_conn.path_db = path_db
db_conn.show_sql_statement = True
db_conn.connect()

In [8]:
# Check if the connection was stablished
db_conn.is_connected()

True

## 2. Load the data into database from csv files

In [9]:
# Create an ETL process object
etl_proc_obj = EtlProcessing(db_conn)

In [10]:
# Crete a file reader object for csv files
csv_reader_obj = FileReaderCsv()  

In [11]:
# Referential Data Source
ref_settings = '../../tests/data/demo/test_ref_sqlite_CLEANED.json'
ref_obj = File(ref_settings)

In [12]:
# Load data from REFERENCIAL
db_ref = etl_proc_obj.load_file_to_db(ref_obj, csv_reader_obj)

2022-08-19 13:23:51,432 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("esg_match_ref")
2022-08-19 13:23:51,437 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-08-19 13:23:51,442 INFO sqlalchemy.engine.Engine PRAGMA main.table_xinfo("esg_match_ref")
2022-08-19 13:23:51,443 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-08-19 13:23:51,447 INFO sqlalchemy.engine.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = ? AND type = 'table'
2022-08-19 13:23:51,448 INFO sqlalchemy.engine.Engine [raw sql] ('esg_match_ref',)
2022-08-19 13:23:51,456 INFO sqlalchemy.engine.Engine PRAGMA main.foreign_key_list("esg_match_ref")
2022-08-19 13:23:51,457 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-08-19 13:23:51,458 INFO sqlalchemy.engine.Engine PRAGMA temp.foreign_key_list("esg_match_ref")
2022-08-19 13:23:51,459 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-08-19 13:23:51,461 INFO sqlalchemy.engine.Engine SELECT sql FROM  (SELECT *

In [13]:
# Target Data Sources
tgt1_settings = '../../tests/data/demo/test_ds1_sqlite_CLEANED.json'
tgt1_obj = File(tgt1_settings)

In [14]:
# Load data from TARGET 1
db_tgt1 = etl_proc_obj.load_file_to_db(tgt1_obj, csv_reader_obj)

2022-08-19 13:23:57,169 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("esg_match_tgt1")
2022-08-19 13:23:57,171 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-08-19 13:23:57,176 INFO sqlalchemy.engine.Engine PRAGMA main.table_xinfo("esg_match_tgt1")
2022-08-19 13:23:57,178 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-08-19 13:23:57,181 INFO sqlalchemy.engine.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = ? AND type = 'table'
2022-08-19 13:23:57,182 INFO sqlalchemy.engine.Engine [raw sql] ('esg_match_tgt1',)
2022-08-19 13:23:57,185 INFO sqlalchemy.engine.Engine PRAGMA main.foreign_key_list("esg_match_tgt1")
2022-08-19 13:23:57,186 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-08-19 13:23:57,187 INFO sqlalchemy.engine.Engine PRAGMA temp.foreign_key_list("esg_match_tgt1")
2022-08-19 13:23:57,188 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-08-19 13:23:57,191 INFO sqlalchemy.engine.Engine SELECT sql FROM  (SEL

In [15]:
# Target Data Sources
tgt2_settings = '../../tests/data/demo/test_ds2_sqlite_CLEANED.json'
tgt2_obj = File(tgt2_settings)

In [16]:
# Load data from TARGET 1
db_tgt2 = etl_proc_obj.load_file_to_db(tgt2_obj, csv_reader_obj)

2022-08-19 13:24:01,342 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("esg_match_tgt2")
2022-08-19 13:24:01,344 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-08-19 13:24:01,349 INFO sqlalchemy.engine.Engine PRAGMA main.table_xinfo("esg_match_tgt2")
2022-08-19 13:24:01,350 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-08-19 13:24:01,353 INFO sqlalchemy.engine.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = ? AND type = 'table'
2022-08-19 13:24:01,354 INFO sqlalchemy.engine.Engine [raw sql] ('esg_match_tgt2',)
2022-08-19 13:24:01,356 INFO sqlalchemy.engine.Engine PRAGMA main.foreign_key_list("esg_match_tgt2")
2022-08-19 13:24:01,357 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-08-19 13:24:01,360 INFO sqlalchemy.engine.Engine PRAGMA temp.foreign_key_list("esg_match_tgt2")
2022-08-19 13:24:01,361 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-08-19 13:24:01,364 INFO sqlalchemy.engine.Engine SELECT sql FROM  (SEL

## 3. Creating matching/no-matching tables

In [17]:
match_settings = '../../tests/data/demo/test_matching_sqlite.json'
no_match_settings = '../../tests/data/demo/test_no_matching_sqlite.json'

In [18]:
file_match = File(match_settings)
file_no_match = File(no_match_settings)

In [19]:
# Call the create_data_source_from_settings() method by passing the File
db_matching = etl_proc_obj.create_data_source(file_match)

2022-08-19 13:24:04,360 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("esg_matching")
2022-08-19 13:24:04,362 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-08-19 13:24:04,368 INFO sqlalchemy.engine.Engine PRAGMA main.table_xinfo("esg_matching")
2022-08-19 13:24:04,369 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-08-19 13:24:04,373 INFO sqlalchemy.engine.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = ? AND type = 'table'
2022-08-19 13:24:04,374 INFO sqlalchemy.engine.Engine [raw sql] ('esg_matching',)
2022-08-19 13:24:04,377 INFO sqlalchemy.engine.Engine PRAGMA main.foreign_key_list("esg_matching")
2022-08-19 13:24:04,378 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-08-19 13:24:04,380 INFO sqlalchemy.engine.Engine PRAGMA temp.foreign_key_list("esg_matching")
2022-08-19 13:24:04,381 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-08-19 13:24:04,382 INFO sqlalchemy.engine.Engine SELECT sql FROM  (SELECT * FROM

In [20]:
# Call the create_data_source_from_settings() method by passing the File
db_no_matching = etl_proc_obj.create_data_source(file_no_match)

2022-08-19 13:24:07,440 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("esg_no_matching")
2022-08-19 13:24:07,441 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-08-19 13:24:07,447 INFO sqlalchemy.engine.Engine PRAGMA main.table_xinfo("esg_no_matching")
2022-08-19 13:24:07,448 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-08-19 13:24:07,452 INFO sqlalchemy.engine.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = ? AND type = 'table'
2022-08-19 13:24:07,453 INFO sqlalchemy.engine.Engine [raw sql] ('esg_no_matching',)
2022-08-19 13:24:07,456 INFO sqlalchemy.engine.Engine PRAGMA main.foreign_key_list("esg_no_matching")
2022-08-19 13:24:07,457 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-08-19 13:24:07,458 INFO sqlalchemy.engine.Engine PRAGMA temp.foreign_key_list("esg_no_matching")
2022-08-19 13:24:07,460 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-08-19 13:24:07,461 INFO sqlalchemy.engine.Engine SELECT sql FROM 

## 4. Checking matching policy and aliases

In [21]:
db_tgt1.get_policy_definition()

{'matching_with_ref1': {'dfm': {'lei': ['LEI']},
  'drm': {'name+country': ['COMPANY', 'COUNTRY']},
  'irm': {'lei': ['LEI'],
   'isin': ['ISIN'],
   'name+country': ['COMPANY', 'COUNTRY']}}}

In [22]:
db_tgt2.get_policy_definition()

{'matching_with_ref1': {'dfm': {'lei': ['LEI']},
  'drm': {'name+country': ['COMPANY', 'COUNTRY']},
  'irm': {'lei': ['LEI'],
   'isin': ['ISIN'],
   'name+country': ['COMPANY', 'COUNTRY']}}}

In [23]:
db_ref.get_mapping_to_alias()

{'LEI': Column('lei_ref', String(), table=<esg_match_ref>),
 'COMPANY': Column('name_ref', String(), table=<esg_match_ref>),
 'COUNTRY': Column('country_ref', String(), table=<esg_match_ref>)}

In [24]:
db_tgt1.get_mapping_to_alias()

{'ISIN': Column('isin_ds1', String(), table=<esg_match_tgt1>),
 'LEI': Column('lei_ds1', String(), table=<esg_match_tgt1>),
 'COMPANY': Column('name_ds1', String(), table=<esg_match_tgt1>),
 'COUNTRY': Column('country_ds1', String(), table=<esg_match_tgt1>)}

In [25]:
db_tgt2.get_mapping_to_alias()

{'ISIN': Column('isin_ds2', String(), table=<esg_match_tgt2>),
 'LEI': Column('lei_ds2', String(), table=<esg_match_tgt2>),
 'SEDOL': Column('sedol_ds2', String(), table=<esg_match_tgt2>),
 'COMPANY': Column('name_ds2', String(), table=<esg_match_tgt2>),
 'COUNTRY': Column('country_ds2', String(), table=<esg_match_tgt2>)}

## 5. Checking attribute mapping between sources and matching tables

In [26]:
db_ref.get_mapping_to_matching()

{'ref_id': Column('id_ref', Integer(), table=<esg_match_ref>, primary_key=True, nullable=False),
 'ref_company': Column('name_ref', String(), table=<esg_match_ref>),
 'ref_country': Column('country_ref', String(), table=<esg_match_ref>)}

In [27]:
db_tgt1.get_mapping_to_matching()

{'tgt_id': Column('id_ds1', Integer(), table=<esg_match_tgt1>, primary_key=True, nullable=False),
 'isin': Column('isin_ds1', String(), table=<esg_match_tgt1>),
 'lei': Column('lei_ds1', String(), table=<esg_match_tgt1>),
 'tgt_company': Column('name_ds1', String(), table=<esg_match_tgt1>),
 'tgt_country': Column('country_ds1', String(), table=<esg_match_tgt1>)}

In [28]:
db_tgt2.get_mapping_to_matching()

{'tgt_id': Column('id_ds2', Integer(), table=<esg_match_tgt2>, primary_key=True, nullable=False),
 'isin': Column('isin_ds2', String(), table=<esg_match_tgt2>),
 'lei': Column('lei_ds2', String(), table=<esg_match_tgt2>),
 'sedol': Column('sedol_ds2', String(), table=<esg_match_tgt2>),
 'tgt_company': Column('name_ds2', String(), table=<esg_match_tgt2>),
 'tgt_country': Column('country_ds2', String(), table=<esg_match_tgt2>)}

## 6. Create policies for matching each target data source with the referential

In [29]:
# Import policy module
from esg_matching.matcher.policy import MatchingPolicy

In [30]:
# Create macthing policy object for target1
policy_match_tgt1 = MatchingPolicy(db_tgt1, 'matching_with_ref1')

In [31]:
# Set the referential and matching/no-matching sources
policy_match_tgt1.set_referential_source(db_ref)
policy_match_tgt1.set_matching_source(db_matching)
policy_match_tgt1.set_no_matching_source(db_no_matching)

In [32]:
# Create macthing policy object for target2
policy_match_tgt2 = MatchingPolicy(db_tgt2, 'matching_with_ref1')

In [33]:
# Set the referential and matching/no-matching sources
policy_match_tgt2.set_referential_source(db_ref)
policy_match_tgt2.set_matching_source(db_matching)
policy_match_tgt2.set_no_matching_source(db_no_matching)

## 7. Perform direct full matching (DFM) for each data source

In [34]:
# Import DFM module
from esg_matching.matcher.dfm import DbMatcherDfm

In [35]:
# Create a matcher object for DFM
dfm_matcher_obj = DbMatcherDfm(db_conn)

In [36]:
# Perform DFM on target 1
dfm_matcher_obj.set_policy(policy_match_tgt1)
dfm_matcher_obj.execute_matching()

2022-08-19 13:24:22,656 INFO sqlalchemy.engine.Engine INSERT INTO esg_matching (ref_name, tgt_name, matching_type, matching_scope, matching_rule, tgt_id, isin, lei, tgt_company, tgt_country, ref_id, ref_company, ref_country) SELECT 'ds_ref' AS ref_name, 'ds_tgt1' AS tgt_name, 'direct' AS matching_type, 'full' AS matching_scope, 'lei' AS matching_rule, esg_match_tgt1.id_ds1 AS tgt_id, esg_match_tgt1.isin_ds1 AS isin, esg_match_tgt1.lei_ds1 AS lei, esg_match_tgt1.name_ds1 AS tgt_company, esg_match_tgt1.country_ds1 AS tgt_country, esg_match_ref.id_ref AS ref_id, esg_match_ref.name_ref AS ref_company, esg_match_ref.country_ref AS ref_country 
FROM esg_match_tgt1 JOIN esg_match_ref ON esg_match_tgt1.lei_ds1 = esg_match_ref.lei_ref
2022-08-19 13:24:22,658 INFO sqlalchemy.engine.Engine [generated in 0.00145s] ()
2022-08-19 13:24:22,664 INFO sqlalchemy.engine.Engine COMMIT
2022-08-19 13:24:22,686 INFO sqlalchemy.engine.Engine INSERT INTO esg_no_matching (tgt_name, tgt_id, isin, lei, tgt_compan

In [37]:
# Perform DFM on target 2
dfm_matcher_obj.set_policy(policy_match_tgt2)
dfm_matcher_obj.execute_matching()

2022-08-19 13:24:22,955 INFO sqlalchemy.engine.Engine INSERT INTO esg_matching (ref_name, tgt_name, matching_type, matching_scope, matching_rule, tgt_id, isin, lei, sedol, tgt_company, tgt_country, ref_id, ref_company, ref_country) SELECT 'ds_ref' AS ref_name, 'ds_tgt2' AS tgt_name, 'direct' AS matching_type, 'full' AS matching_scope, 'lei' AS matching_rule, esg_match_tgt2.id_ds2 AS tgt_id, esg_match_tgt2.isin_ds2 AS isin, esg_match_tgt2.lei_ds2 AS lei, esg_match_tgt2.sedol_ds2 AS sedol, esg_match_tgt2.name_ds2 AS tgt_company, esg_match_tgt2.country_ds2 AS tgt_country, esg_match_ref.id_ref AS ref_id, esg_match_ref.name_ref AS ref_company, esg_match_ref.country_ref AS ref_country 
FROM esg_match_tgt2 JOIN esg_match_ref ON esg_match_tgt2.lei_ds2 = esg_match_ref.lei_ref
2022-08-19 13:24:22,956 INFO sqlalchemy.engine.Engine [generated in 0.00130s] ()
2022-08-19 13:24:22,976 INFO sqlalchemy.engine.Engine COMMIT
2022-08-19 13:24:22,999 INFO sqlalchemy.engine.Engine INSERT INTO esg_no_matchin

## 8. Perform direct residual matching (DRM) for each data source

In [38]:
# Import DRM module
from esg_matching.matcher.drm import DbMatcherDrm

In [39]:
# Create a matcher object for DRM
drm_matcher_obj = DbMatcherDrm(db_conn)

In [40]:
# Perform DRM on target 1
drm_matcher_obj.set_policy(policy_match_tgt1)
drm_matcher_obj.execute_matching()

2022-08-19 13:24:24,606 INFO sqlalchemy.engine.Engine INSERT INTO esg_matching (ref_name, tgt_name, matching_type, matching_scope, matching_rule, tgt_id, isin, lei, tgt_company, tgt_country, ref_id, ref_company, ref_country) SELECT 'ds_ref' AS ref_name, 'ds_tgt1' AS tgt_name, 'direct' AS matching_type, 'residual' AS matching_scope, 'name+country' AS matching_rule, esg_no_matching.tgt_id, esg_no_matching.isin, esg_no_matching.lei, esg_no_matching.tgt_company, esg_no_matching.tgt_country, esg_match_ref.id_ref AS ref_id, esg_match_ref.name_ref AS ref_company, esg_match_ref.country_ref AS ref_country 
FROM esg_no_matching JOIN esg_match_ref ON esg_no_matching.tgt_company = esg_match_ref.name_ref AND esg_no_matching.tgt_country = esg_match_ref.country_ref 
WHERE esg_no_matching.tgt_name = ?
2022-08-19 13:24:24,607 INFO sqlalchemy.engine.Engine [generated in 0.00125s] ('ds_tgt1',)
2022-08-19 13:24:24,611 INFO sqlalchemy.engine.Engine COMMIT
2022-08-19 13:24:24,649 INFO sqlalchemy.engine.Engi

In [41]:
# Perform DRM on target 2
drm_matcher_obj.set_policy(policy_match_tgt2)
drm_matcher_obj.execute_matching()

2022-08-19 13:24:24,974 INFO sqlalchemy.engine.Engine INSERT INTO esg_matching (ref_name, tgt_name, matching_type, matching_scope, matching_rule, tgt_id, isin, lei, sedol, tgt_company, tgt_country, ref_id, ref_company, ref_country) SELECT 'ds_ref' AS ref_name, 'ds_tgt2' AS tgt_name, 'direct' AS matching_type, 'residual' AS matching_scope, 'name+country' AS matching_rule, esg_no_matching.tgt_id, esg_no_matching.isin, esg_no_matching.lei, esg_no_matching.sedol, esg_no_matching.tgt_company, esg_no_matching.tgt_country, esg_match_ref.id_ref AS ref_id, esg_match_ref.name_ref AS ref_company, esg_match_ref.country_ref AS ref_country 
FROM esg_no_matching JOIN esg_match_ref ON esg_no_matching.tgt_company = esg_match_ref.name_ref AND esg_no_matching.tgt_country = esg_match_ref.country_ref 
WHERE esg_no_matching.tgt_name = ?
2022-08-19 13:24:24,976 INFO sqlalchemy.engine.Engine [generated in 0.00146s] ('ds_tgt2',)
2022-08-19 13:24:24,980 INFO sqlalchemy.engine.Engine COMMIT
2022-08-19 13:24:25,0

## 9. Perform indirect full matching (IFM) for each data source

In [42]:
# Import IFM module
from esg_matching.matcher.irm import DbMatcherIrm

In [43]:
# Create a matcher object for IFM
irm_matcher_obj = DbMatcherIrm(db_conn)

In [44]:
# Perform IFM on target 1
irm_matcher_obj.set_policy(policy_match_tgt1)
irm_matcher_obj.execute_matching()

2022-08-19 13:24:26,642 INFO sqlalchemy.engine.Engine INSERT INTO esg_matching (matching_type, matching_scope, matching_rule, tgt_name, tgt_id, tgt_company, tgt_country, isin, lei, sedol, ref_name, ref_id, ref_company, ref_country) SELECT 'indirect' AS matching_type, 'full' AS matching_scope, 'lei' AS matching_rule, esg_no_matching.tgt_name, esg_no_matching.tgt_id, esg_no_matching.tgt_company, esg_no_matching.tgt_country, esg_no_matching.isin, esg_no_matching.lei, esg_no_matching.sedol, esg_matching.tgt_name AS ref_name, esg_matching.tgt_id AS ref_id, esg_matching.tgt_company AS ref_company, esg_matching.tgt_country AS ref_country 
FROM esg_no_matching JOIN esg_matching ON esg_no_matching.lei = esg_matching.lei 
WHERE esg_no_matching.tgt_name = ? AND esg_matching.tgt_name != ?
2022-08-19 13:24:26,643 INFO sqlalchemy.engine.Engine [generated in 0.00127s] ('ds_tgt1', 'ds_tgt1')
2022-08-19 13:24:26,646 INFO sqlalchemy.engine.Engine COMMIT
2022-08-19 13:24:26,650 INFO sqlalchemy.engine.Eng

In [45]:
# Perform IFM on target 2
irm_matcher_obj.set_policy(policy_match_tgt2)
irm_matcher_obj.execute_matching()

2022-08-19 13:24:29,346 INFO sqlalchemy.engine.Engine INSERT INTO esg_matching (matching_type, matching_scope, matching_rule, tgt_name, tgt_id, tgt_company, tgt_country, isin, lei, sedol, ref_name, ref_id, ref_company, ref_country) SELECT 'indirect' AS matching_type, 'full' AS matching_scope, 'lei' AS matching_rule, esg_no_matching.tgt_name, esg_no_matching.tgt_id, esg_no_matching.tgt_company, esg_no_matching.tgt_country, esg_no_matching.isin, esg_no_matching.lei, esg_no_matching.sedol, esg_matching.tgt_name AS ref_name, esg_matching.tgt_id AS ref_id, esg_matching.tgt_company AS ref_company, esg_matching.tgt_country AS ref_country 
FROM esg_no_matching JOIN esg_matching ON esg_no_matching.lei = esg_matching.lei 
WHERE esg_no_matching.tgt_name = ? AND esg_matching.tgt_name != ?
2022-08-19 13:24:29,347 INFO sqlalchemy.engine.Engine [cached since 2.705s ago] ('ds_tgt2', 'ds_tgt2')
2022-08-19 13:24:29,351 INFO sqlalchemy.engine.Engine COMMIT
2022-08-19 13:24:29,370 INFO sqlalchemy.engine.E

## 10. Saving the matching table as .csv file

In [46]:
import pandas as pd

In [47]:
df_matching = db_matching.get_data_as_df()

2022-08-19 13:24:30,798 INFO sqlalchemy.engine.Engine SELECT esg_matching.timestamp, esg_matching.matching_id, esg_matching.ref_name, esg_matching.tgt_name, esg_matching.matching_type, esg_matching.matching_scope, esg_matching.matching_rule, esg_matching.ref_id, esg_matching.ref_company, esg_matching.ref_country, esg_matching.tgt_id, esg_matching.tgt_company, esg_matching.tgt_country, esg_matching.isin, esg_matching.lei, esg_matching.sedol 
FROM esg_matching
2022-08-19 13:24:30,799 INFO sqlalchemy.engine.Engine [generated in 0.00126s] ()


In [48]:
df_matching

,timestamp,matching_id,ref_name,tgt_name,matching_type,matching_scope,matching_rule,ref_id,ref_company,ref_country,tgt_id,tgt_company,tgt_country,isin,lei,sedol
0,2022-08-19 12:24:22,1,ds_ref,ds_tgt1,direct,full,lei,3,STARCOURT MALL SRLS.,it,4,STARCOURT MALL SOCIETÀ A RESPONSABILITÀ LIMITA...,it,None,8156006CE62347C74658,None
1,2022-08-19 12:24:22,2,ds_ref,ds_tgt2,direct,full,lei,3,STARCOURT MALL SRLS.,it,2,STARCOURT MALL,it,NO0003058109,8156006CE62347C74658,10009110965
2,2022-08-19 12:24:22,3,ds_ref,ds_tgt2,direct,full,lei,6,INGEN SOCIÉTÉ COOPÉRATIVE DE PRODUCTION,fr,4,INGEN,fr,FR0000072910,9695001UE8RNVNTE9L89,2019952
3,2022-08-19 12:24:22,4,ds_ref,ds_tgt2,direct,full,lei,4,STERLING COOPER,gb,6,STERLING COOPER,gb,GB00B1YW4409,213800KY4C9WU7WBW518,B1YW440
4,2022-08-19 12:24:24,5,ds_ref,ds_tgt1,direct,residual,name+country,1,SALLMOOR AGRUPACION DE INTERES ECONOMICO,es,1,SALLMOOR AGRUPACION DE INTERES ECONOMICO,es,SK1120005824,097900BHK10000084115,None
5,2022-08-19 12:24:24,6,ds_ref,ds_tgt1,direct,residual,name+country,2,HONEYDUKES LIMITED,us,3,HONEYDUKES LIMITED,us,None,254900B1P3S786KDAW57,None
6,2022-08-19 12:24:24,7,ds_ref,ds_tgt2,direct,residual,name+country,8,SPECTRE EMPRESA INDIVIDUAL DE RESPONSABILIDADE...,br,5,SPECTRE EMPRESA INDIVIDUAL DE RESPONSABILIDADE...,br,BRCIELACNOR3,None,2046853
7,2022-08-19 12:24:29,8,ds_tgt1,ds_tgt2,indirect,full,lei,1,SALLMOOR AGRUPACION DE INTERES ECONOMICO,es,1,HONEYDUKES LIMITED,None,None,097900BHK10000084115,7108899
8,2022-08-19 12:24:29,9,ds_tgt1,ds_tgt2,indirect,full,isin,1,SALLMOOR AGRUPACION DE INTERES ECONOMICO,es,7,CENTRAL PERK,sk,SK1120005824,None,B1YW440


In [49]:
df_matching.to_csv('my_matching.csv', index=False)

In [50]:
# Import the module for connection to trino database
from esg_matching.engine.connectors.trino import TrinoConnector

## 11. Close database connection

In [51]:
db_conn.disconnect()

In [52]:
db_conn.is_connected()

False